# GPT batch script

This routine shows an example of how to use python to call SNAP's GPT processor, to allow automated processing of S3A OLCI L1 images from the command line.

    Version: 1.1
    Author: B Loveday, PML
    Notes:
    1. A batch scripting version of the same file, batch_gpt.py, can be found in the same folder.
    2. Both this script, and the batch version, will require you to adapt the paths to the GPT executable and data
       on your local system.
      

We begin by importing some python modules

In [1]:
import os, shutil
from os import path, listdir, system
import fnmatch
import fileinput
import subprocess

Next we have to tell our code where the GPT executable is. The default path is hown below, but you may have to adapt this depending on where you installed SNAP.

In [2]:
# CONFIG
# set up the environment, path to your gpt processor
gptProcessor = '/Users/jconchas/Applications/snap/bin/gpt'

Now we must point the code to our config file (in the same directory as this file), and the data we downloaded. It is the configuration file that describes the processes that SNAP will perform in our automated chain.

In [3]:
# define your graph xml, and input and output directory paths
template_xml = path.join(os.getcwd(),'config_template.xml')
input_dir = path.join(os.getcwd(),'GPT_L1_input')
output_dir = path.join(os.getcwd(),'GPT_L2_output')
print(input_dir)
print(output_dir)

/Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L1_input
/Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L2_output


Now we search the input directory to find Level-1 Sentinel-3 data

In [4]:
# INPUTS
# make a list of all the input files in your input directory
input_files=[]
for root, _, filenames in os.walk(input_dir):
    for filename in fnmatch.filter(filenames, '*xfdumanifest.xml'):
        input_files.append(os.path.join(root, filename))
        
print(input_files)        

['/Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L1_input/S3A_OL_1_EFR____20181003T014336_20181003T014636_20181003T034142_0179_036_231_2340_MAR_O_NR_002.SEN3/xfdumanifest.xml']


And finally, we start the main part of our code, where we call GPT to process this L1 data. 

In [5]:
# MAIN
# this is the loop where the magic happens :-)
# the loop goes over each input file in the input_files list
for input_file in input_files:
    file_tag = os.path.basename(os.path.dirname(input_file))
    file_tag = file_tag.replace('.SEN3','')
    output_file = input_file.replace(input_dir,output_dir)
    output_file = output_file.replace('.SEN3/xfdumanifest.xml','_SUBSET_IDEPIX_C2RCC.nc')
    # the output file is named from the input file with the _C2RCCC suffix.
    # Here you will write a netcdf file. this can be changed
    print('Processing: '+str(input_file))
    print('To: '+str(output_file))
    
    # now adapt the config xml file for this specific input_file
    my_config = input_file.replace('xfdumanifest.xml','run_config.xml')
    if os.path.exists(my_config):
        os.remove(my_config)

    print('Generating config: '+str(my_config))
    shutil.copy(template_xml,my_config)

    print('Adapting config: '+str(my_config))
    
    # Read in the file
    with open(my_config, 'r') as file :
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('SOURCE_PRODUCT', input_file)
    # Write the file out again
    with open(my_config, 'w') as file:
        file.write(filedata)

    # Read in the file
    with open(my_config, 'r') as file :
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('OUTPUT_PRODUCT', output_file)
    # Write the file out again
    with open(my_config, 'w') as file:
        file.write(filedata)
        
    # the processing call is a follows below.
    c2rcc_processingCall = gptProcessor + ' ' + my_config

    # useful to check that the command call is correct before launching the call (comment / uncomment the next line)
    print('Config ready; launching:')
    print(c2rcc_processingCall)

    # python call, uncomment when the printed call satisfies your requirements
    system(c2rcc_processingCall)
print('Done!')

Processing: /Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L1_input/S3A_OL_1_EFR____20181003T014336_20181003T014636_20181003T034142_0179_036_231_2340_MAR_O_NR_002.SEN3/xfdumanifest.xml
To: /Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L2_output/S3A_OL_1_EFR____20181003T014336_20181003T014636_20181003T034142_0179_036_231_2340_MAR_O_NR_002_SUBSET_IDEPIX_C2RCC.nc
Generating config: /Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L1_input/S3A_OL_1_EFR____20181003T014336_20181003T014636_20181003T034142_0179_036_231_2340_MAR_O_NR_002.SEN3/run_config.xml
Adapting config: /Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L1_input/S3A_OL_1_EFR____20181003T014336_20181003T014636_20181003T034142_0179_036_231_2340_MAR_O_NR_002.SEN3/run_config.xml
Config ready; launching:
/Users/jconchas/Applications/snap/bin/gpt /Users/jconchas/Documents/Research/Sentinel_3/python-examples/GPT/GPT_L1_input/S3A_OL_1_EFR____2018